In [1]:
from dfsubset import *
from models import *

from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn import metrics

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#df = pd.read_csv('stopah.csv')
#df, X, y, X_train, X_test, y_train, y_test = dataset(df,'Lille','yes','cc','train-test',0.25)

In [3]:
subset = ['baseline','7day','Lille','full']

treatment = ['yes','no']

score_yes = []
auc_yes = []
score_no = []
auc_no = []
ebm_auc_yes = []
score_ebm_yes= []
ebm_auc_no = []
score_ebm_no = []

In [4]:
for t in ['yes','no']:
    for s in subset:
        xgb_score, xgb_auc = xgb_results(s,t,'train-test')
        ebm_score, ebm_auc = ebm_results(s,t,'train-test')
        
        #print(auc_score)
        if t == 'yes':
            score_yes.append(xgb_score)
            auc_yes.append(xgb_auc)
            score_ebm_yes.append(ebm_score)
            ebm_auc_yes.append(ebm_auc)
        elif t== 'no':
            score_no.append(xgb_score)
            auc_no.append(xgb_auc)
            score_ebm_no.append(ebm_score)
            ebm_auc_no.append(ebm_auc)
            

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lilimati

In [5]:
score_mixed = []
auc_mixed = []

ebm_score_mixed = []
ebm_auc_mixed = []

for s in subset:
        xgb_score, auc_score = xgb_results(s,'none','mixed')
        ebm_score, ebm_auc = ebm_results(s,'none','mixed')
        score_mixed.append(xgb_score)
        auc_mixed.append(auc_score)
        ebm_score_mixed.append(ebm_score)
        ebm_auc_mixed.append(ebm_auc)

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lilimati

In [6]:
results = pd.DataFrame({'Subset':subset,
                        'Treated XGB AUC':auc_yes,
                        'Untreated XGB AUC':auc_no,
                        'Cross XGB AUC': auc_mixed,
                        'Treated XGB Score':score_yes,
                        'Untreated XGB Score':score_no,
                        'Mixed XGB Score': auc_mixed,
                        'Treated EBM AUC': ebm_auc_yes,
                        'Untreated EBM AUC': ebm_auc_no,
                        'Treated EBM Score': score_ebm_yes,
                        'Untreated EBM Score': score_ebm_no
                       })


In [7]:
results

,Subset,Treated XGB AUC,Untreated XGB AUC,Cross XGB AUC,Treated XGB Score,Untreated XGB Score,Mixed XGB Score,Treated EBM AUC,Untreated EBM AUC,Treated EBM Score,Untreated EBM Score
0,baseline,0.776316,0.755639,0.747814,80.43,84.44,0.747814,78.62,80.83,82.61,84.44
1,7day,0.749373,0.745597,0.770013,79.49,86.21,0.770013,79.45,71.72,76.92,79.31
2,Lille,0.645646,0.678133,0.705146,72.28,70.18,0.705146,63.86,71.32,71.29,74.56
3,full,0.642857,0.830000,0.678871,70.37,78.79,0.678871,70.00,84.00,74.07,75.76


In [8]:
#Add Lille score

lille_yes = []
lille_no = []

In [9]:
for t in ['yes','no']: 
    print(t)
    df = pd.read_csv('stopah.csv')
    df, X, y, X_train, X_test, y_train, y_test = dataset(df,'Lille',t,'cc','train-test',0.3)
    Lille = 3.19 - 0.101 * X_test['Age.at.randomisation..calc.']  + 0.147 * X_test['Albumin...Merged'] + \
0.0165 * (X_test['Bilirubin.day.7'] -X_test['Bilirubin.Merged']) \
-0.206 * X_test['Renal Insufficency'] \
- 0.0065 * X_test['Bilirubin.Merged'] \
- 0.0096 * X_test['Prothrombin.Time..patient....Merged'] 
    prop = 1/(1+np.exp(-Lille))
    fpr, tpr, thresholds = roc_curve(y_test, prop, pos_label=1)
    #
    if t == 'yes':
        lille_yes.append(metrics.auc(fpr, tpr))
    elif t == 'no':
        lille_no.append(metrics.auc(fpr, tpr))
        
        

yes


/Users/lilimatic/stopah/dfsubset.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Renal Insufficency'] = (df['Creatinine..mg.dL....Merged'].loc[:] > 1.3).astype('float')


no


/Users/lilimatic/stopah/dfsubset.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Renal Insufficency'] = (df['Creatinine..mg.dL....Merged'].loc[:] > 1.3).astype('float')


In [10]:
lille_auc_mixed = []

df = pd.read_csv('stopah.csv')
df, X, y, X_train, X_test, y_train, y_test = dataset(df,'Lille','none','cc','mixed',0.3)

Lille = 3.19 - 0.101 * X_test['Age.at.randomisation..calc.']  + 0.147 * X_test['Albumin...Merged'] + \
0.0165 * (X_test['Bilirubin.Merged']-X_test['Bilirubin.day.7'] -) \
-0.206 * X_test['Renal Insufficency'] \
- 0.0065 * X_test['Bilirubin.Merged'] \
- 0.0096 * X_test['Prothrombin.Time..patient....Merged'] 

prop = 1/(1+np.exp(-Lille))
fpr, tpr, thresholds = roc_curve(y_test, prop, pos_label=1)

lille_mixed = [metrics.auc(fpr, tpr)]
        
        

SyntaxError: invalid syntax (<ipython-input-10-c63d6a727ec5>, line 7)

In [ ]:
lille_yes

In [ ]:
lille_no

In [ ]:
llille_mixed